In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import eli5
import shap
from IPython.display import HTML
import json
import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [3]:
# with pd.HDFStore('./hdf/Store.h5',complib='blosc:blosclz') as store:
#     categorial_features = list(store['initial/categorial_features'].values.flatten())
#     numerical_features = list(store['initial/numerical_features'].values.flatten())
categorial_features = json.load(open('pickles/categorial_features.json','r'))
numerical_features = json.load(open('pickles/numerical_features.json','r'))

In [4]:
from BayDS import *


In [5]:
with pd.HDFStore('./hdf/pre-aggregate.h5') as store:
    data = store[f'data']


KeyError: 'No object named data in the file'

In [28]:
from BayDS.io import * 
data = reduce_mem_usage(data)
gc.collect()

Mem. usage decreased to 822.71 Mb (67.3% reduction)


3787

In [29]:
with pd.HDFStore('./hdf/pre-aggregate.h5') as store:
    store[f'data'] = data

In [30]:
score = quick_score2(data,categorial_features,drop_columns=['Date','card_id'])
print ("Main score is ", score)

Fold 5 started at Sun Aug 25 14:06:29 2019
CV mean score: 0.9168, std: 0.0000.
Main score is  0.9168228074769871


# Categorial Aggregates

In [65]:
def add_categorial_aggregate(df, target_column, group_column):
    print(target_column, group_column)
    data_slice = df[[group_column,target_column]]
    gb = data_slice.groupby([group_column,target_column])[target_column].transform('count') /  data_slice.groupby([group_column])[target_column].transform('count')
    return gb
    
#     if scoring:
#         new_score = quick_score2(df,categorial_features,addition=to_mean,categorial_features_in_addition=[])
#         delta_score = new_score - score
#         with open('./hdf/num_agg_scores.csv','a') as f:
#             f.write(f'{target_column}_to_mean_{group_column}, {delta_score}\n')
#     if save:
#         with pd.HDFStore('./hdf/engineering.h5') as store:
#             store[f'numerical/{target_column}_to_std_{group_column}'] = to_mean[[f'{target_column}_to_mean_{group_column}']]

#     to_std = pd.DataFrame()
#     to_std[f'{target_column}_to_std_{group_column}'] = df[target_column] / data.groupby([group_column])[target_column].transform('std')
#     if scoring:
#         new_score = quick_score2(df,categorial_features,addition=to_std,categorial_features_in_addition=[])
#         delta_score = new_score - score
#         with open('./hdf/num_agg_scores.csv','a') as f:
#             f.write(f'{target_column}_to_std_{group_column}, {delta_score}\n')
#     if save:
#         with pd.HDFStore('./hdf/engineering.h5') as store:
#             store[f'numerical/{target_column}_to_std_{group_column}'] = to_std[[f'{target_column}_to_std_{group_column}']]
#     return to_mean.join(to_std)
    
# add_numerical_aggregate(data,'TransactionAmt', 'card_id')
# add_numerical_aggregate(data,'dist1', 'card_id')
# add_numerical_aggregate(data,'_Hours', 'card_id')





In [68]:
cat_aggs = pd.DataFrame(index=data.index)
# started = False
for cat_feature in categorial_features:
#     if not started and num_feature != 'V1':
#         continue
#     started = True
    for group_feature in ['card_id']:
        try:
            cat_aggs[f'{cat_feature}_freq_groupby_{group_feature}']=add_categorial_aggregate(data,cat_feature, group_feature)
        except Exception as ex:
            print('Exception', ex)
            

id_13 card_id
id_14 card_id
id_17 card_id
id_18 card_id
id_19 card_id
id_20 card_id
id_21 card_id
id_22 card_id
id_23 card_id
id_24 card_id
id_25 card_id
id_26 card_id
id_27 card_id
id_30 card_id
Exception "['id_30'] not in index"
id_31 card_id
Exception "['id_31'] not in index"
id_33 card_id
DeviceType card_id
DeviceInfo card_id
P_emaildomain card_id
Exception "['P_emaildomain'] not in index"
R_emaildomain card_id
Exception "['R_emaildomain'] not in index"
card1 card_id
card2 card_id
card3 card_id
card5 card_id
addr1 card_id
addr2 card_id
P_emaildomain_bin card_id
P_emaildomain_suffix card_id
R_emaildomain_bin card_id
R_emaildomain_suffix card_id
M1 card_id
Exception "['M1'] not in index"
M2 card_id
M3 card_id
M4 card_id
M5 card_id
M6 card_id
M7 card_id
M8 card_id
M9 card_id
card4 card_id
card6 card_id
ProductCD card_id
id_12 card_id
id_15 card_id
id_16 card_id
id_28 card_id
id_29 card_id
id_32 card_id
id_34 card_id
id_35 card_id
id_36 card_id
id_37 card_id
id_38 card_id


In [69]:
cat_aggs.head()

,id_13_freq_groupby_card_id,id_14_freq_groupby_card_id,id_17_freq_groupby_card_id,id_18_freq_groupby_card_id,id_19_freq_groupby_card_id,id_20_freq_groupby_card_id,id_21_freq_groupby_card_id,id_22_freq_groupby_card_id,id_23_freq_groupby_card_id,id_24_freq_groupby_card_id,...,id_15_freq_groupby_card_id,id_16_freq_groupby_card_id,id_28_freq_groupby_card_id,id_29_freq_groupby_card_id,id_32_freq_groupby_card_id,id_34_freq_groupby_card_id,id_35_freq_groupby_card_id,id_36_freq_groupby_card_id,id_37_freq_groupby_card_id,id_38_freq_groupby_card_id
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,...,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000
2987001,0.929216867469880,0.914909638554217,0.914156626506024,0.966867469879518,0.914156626506024,0.914156626506024,0.993222891566265,0.993222891566265,0.993222891566265,0.993222891566265,...,0.914156626506024,0.914156626506024,0.914156626506024,0.914156626506024,0.915662650602410,0.915662650602410,0.914156626506024,0.914156626506024,0.914156626506024,0.914156626506024
2987002,0.996082820369334,0.994963626189144,0.994963626189144,0.998880805819810,0.994963626189144,0.994963626189144,0.999440402909905,0.999440402909905,0.999440402909905,0.999440402909905,...,0.994404029099049,0.994963626189144,0.994404029099049,0.994404029099049,0.994963626189144,0.994404029099049,0.994404029099049,0.994404029099049,0.994404029099049,0.994404029099049
2987003,0.947908445146014,0.919757958431992,0.920021047092870,0.969481715338069,0.920021047092870,0.920021047092870,0.995527492765062,0.995527492765062,0.995527492765062,0.995527492765062,...,0.918047882136280,0.919363325440674,0.918179426466719,0.918179426466719,0.919757958431992,0.921468034727703,0.918047882136280,0.918047882136280,0.918047882136280,0.918047882136280
2987004,0.833333333333333,0.066666666666667,0.333333333333333,0.900000000000000,0.066666666666667,0.033333333333333,1.000000000000000,1.000000000000000,1.000000000000000,1.000000000000000,...,0.166666666666667,0.166666666666667,0.133333333333333,0.133333333333333,0.066666666666667,0.333333333333333,0.333333333333333,0.266666666666667,0.266666666666667,0.300000000000000


In [70]:
cat_aggs = cat_aggs.astype(np.float32)

In [71]:
with pd.HDFStore('./hdf/categorial_aggregations.h5',complib='blosc:blosclz') as store:
    store.put('cat_aggs', cat_aggs)
    

0) Treat categorial features accurately, find very diverse and unite them

1) Убрать коррелирующие фичи

2) Убрать ту из них что даёт меньший джини

3) TimeKFold instead of StratifiedKFold